In [1]:
import pymysql
import numpy as np
import pandas as pd
from ckip_transformers.nlp import CkipWordSegmenter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


c:\Users\Takodachi\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("======輸入查詢的公司代碼======")
company_code = "2330"
print(company_code)
print("======輸入查詢年度======")
year = "111"
print(year)

======輸入查詢的公司代碼======
2330
======輸入查詢年度======
111


In [3]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='stockdb',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

In [4]:
def query_by_id(id):
    try:
        # 使用 cursor 建立執行 SQL 的物件
        with connection.cursor() as cursor:
            # 執行 SQL 查詢
            sql = "SELECT * FROM quarterly_reports WHERE company_code = %s"
            cursor.execute(sql, (id,))
            result = cursor.fetchall()

            if result:
                return pd.DataFrame(result)
            else:
                print("找不到該 ID 的資料")

    except pymysql.Error as e:
        print(f"查詢發生錯誤：{e}")

    # finally:
    #     # 關閉連線
    #     connection.close()

df = query_by_id(company_code)

In [5]:
quarters = ["01","02","03","04"]
years = [year]
result = df[df['quarter_date'].isin(quarters) & df['year'].isin(years)]

In [6]:
import re
def clear(content):
    result_str = ''
    for word in content:
        if word == '\u3000' or word == '\n':
            continue
        result_str += word
    
    # 移除標點符號
    result_str = re.sub(r'[^\w\s]', '', result_str)
    # 移除空格
    result_str = re.sub(r'\s+', '', result_str)

    return result_str

In [7]:
result.loc[:, 'content'] = result['content'].apply(clear)
ws_driver  = CkipWordSegmenter(model="bert-base")
def word_segmentation(content):
    words = ws_driver([content])[0]
    return ' '.join(words)

# 對 content 欄位進行斷詞
result.loc[:, 'content'] = result['content'].apply(word_segmentation)

Inference: 100%|██████████| 1/1 [00:06<00:00,  6.58s/it]


In [8]:
t1 = ' '.join(result.loc[result['quarter_date'] == "01", 'content'])
t2 = ' '.join(result.loc[result['quarter_date'] == "02", 'content'])
t3 = ' '.join(result.loc[result['quarter_date'] == "03", 'content'])
t4 = ' '.join(result.loc[result['quarter_date'] == "04", 'content'])

In [9]:
def Cosine(x, y):
    documents = [x, y]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity_score = cosine_similarity(tfidf_matrix)[0, 1]
    print(f"相似度分數: {round(similarity_score * 100, 2)} %")

ct21 = Cosine(t2, t1)
ct32 = Cosine(t3, t2)
ct43 = Cosine(t4, t3)

相似度分數: 86.88 %
相似度分數: 91.27 %
相似度分數: 53.99 %


TODO : 視覺化、機器學習的實驗